### prepare SSAGA for the NDA - download data (sans restricted data) from PreRelease folder

In [ ]:
import os, datetime
import csv
import sys
import shutil
from openpyxl import load_workbook
import pandas as pd
from io import BytesIO
import numpy as np
import subprocess
from scipy import stats
from ccf.box import LifespanBox
from ccf.config import LoadSettings
from ccf.redcap import RedcapTable 
import Crosswalk.cleanupfuncs as cleanupfuncs
config = LoadSettings()
eventlist=['V1','V2']

In [ ]:
box=LifespanBox(cache='./')
verbose = True
snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
pathout="./prepped/hca" 

#Rosetta (a.k.a Inventory) file will have all the nda vars and pedids
extrainfo=config['rosetta']['hca']['filename']

#redcapconfigfile="/home/petra/Behavioral/Lifespan/PreRelease/PreRelease/.boxApp/redcapconfig.csv"

snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
#catfromdate=max of last run--'2019-06-17'
print(snapshotdate)
print(extrainfo)

In [ ]:
handle=[1143375281350]
ssagad=box.download_files(handle)
dir(ssagad)

In [ ]:
inventory=pd.read_csv(extrainfo)
print('Inventory Shape out of the Box',inventory.shape)

inventory=inventory.loc[inventory.IntraDB != "CCF_PCMP_ITK"][['subject','pseudo_guid','nda_interview_date','nda_age','M/F','pedid','redcap_event_name','redcap_event']].copy()
print('Inventory Shape out of the Box',inventory.shape)

inventory=inventory.loc[inventory.nda_age.isnull()==False]
print('Inventory Shape out of the Box',inventory.shape)

inventory.nda_interview_date=pd.to_datetime(inventory.nda_interview_date).dt.strftime('%m/%d/%Y')
inventory.nda_age=inventory.nda_age.round(0).astype(int)
print('Inventory Shape out of the Box',inventory.shape)
print('Number of Unique Subjects',len(inventory.subject.unique()))

inventory=inventory.loc[inventory.redcap_event.isin(eventlist)]
inventory.shape


In [ ]:
datafile='HCA_SSAGA_2023-02-17.csv'
ssagadata=pd.read_csv(datafile,low_memory=False)
print(ssagadata.shape)

In [ ]:
noteslist=[]
specifylist=[]

for i in list(ssagadata.columns):
    if 'notes' in i:
        print(i)
        noteslist=noteslist+[i]
#print(noteslist)

for i in list(ssagadata.columns):
    if 'specify' in i:
        print(i)
        specifylist=specifylist+[i]
#print(specifylist)


In [ ]:
ssagadata=ssagadata.drop(columns=['hcpa_id','flagged','study','register_subject_complete','vist1','vist2','register_visit_1_complete','register_visit_2_complete','redcap_event_name','ssaga_complete'])
#ssagadata=ssagadata.drop(columns=noteslist)
#ssagadata=ssagadata.drop(columns=specifylist)

ssagadropsubs=pd.merge(ssagadata, inventory, on=['subject','redcap_event'], how='right').drop(columns=['pedid', 'redcap_event_name'])
ssagadropsubs.shape

In [ ]:
ssagas=ssagadropsubs.rename(columns={'subject':'src_subject_id','M/F':'sex','nda_age':'interview_age','nda_interview_date':'interview_date','pseudo_guid':'subjectkey', 'redcap_event':'event'})
ssagas=ssagas.rename(columns={'dr1___1':'dr1_1','dr1___2':'dr1_2','dr1___3':'dr1_3','dr1___4':'dr1_4','dr1___5':'dr1_5','dr1___6':'dr1_6','dr1___7':'dr1_7','dr1___8':'dr1_8','dr1___9':'dr1_09'})


In [ ]:
#import map
mapfile=pd.read_excel('./Mapping Key.xlsx', sheet_name='HCP_SSAGA_IV_15Dec2021')
mapfile['SSAGA Element Name'].head(20)

listnames=list(mapfile['SSAGA Element Name'].str.strip())

for i in listnames:
    if i not in list(ssagas.columns):
        print(i,'is not in ssaga data')


In [ ]:
#import options
pd.options.display.max_colwidth=100

print(mapfile.loc[mapfile['SSAGA Element Name']=='dr1'][['Notes']])
##ssagas[['dr1']]#.columns

#dr1_list = ['dr1___1','dr1___2','dr1___3','dr1___4','dr1___5','dr1___6','dr1___7','dr1___8','dr1___9']
elements_list = ['dr1_1','dr1_2','dr1_3','dr1_4','dr1_5','dr1_6','dr1_7','dr1_8','dr1_9']
dr1 = ssagas[elements_list].sum(axis=1).value_counts()
#for COC,STIM,SED,OPIATE,PCP,HAL,SOL,COMB and OTHER

In [ ]:
#split into the two structures
#part1 AND part2 are the ndar mandatory variables
mapfile.columns
mapfile.head()
mapfile['Short Name'].value_counts()

In [ ]:
part1=mapfile.loc[mapfile['Short Name'].str.contains('ssaga_part101')]
part2=mapfile.loc[mapfile['Short Name'].str.contains('ssaga_part201')]

In [ ]:
part1['Short Name'].value_counts()
part2['Short Name'].value_counts()

In [ ]:
part1['Short Name'].head()
part1.head()

In [ ]:
part1.head()
part1b=part1.copy()
#part1b=part1.loc[~(part1['SSAGA Element Name'].isin(noteslist))]
#part1b=part1b.loc[~(part1['SSAGA Element Name'].isin(specifylist))]
struct101=ssagas[list(part1b['SSAGA Element Name'])] #.head()
rename1=dict(zip(list(part1b['SSAGA Element Name']),list(part1b['NDA Element Name'])))

#remove dr1 because export uses dummy vars.
part2b=part2.loc[~(part2['SSAGA Element Name']=='dr1')]
#part2b=part2b.loc[~(part2b['SSAGA Element Name'].isin(noteslist))]
#part2b=part2b.loc[~(part2b['SSAGA Element Name'].isin(specifylist))]

struct201=ssagas[list(part2b['SSAGA Element Name'])]#+dr1_list]
#rename2=dict(zip(list(part2b['SSAGA Element Name'])+dr1_list,list(part2b['NDA Element Name'])+elements_list))
rename2=dict(zip(list(part2b['SSAGA Element Name']),list(part2b['NDA Element Name'])))

In [ ]:
len(list(part2b['NDA Element Name']))
len(list(part2b['SSAGA Element Name']))

In [ ]:
ready1=struct101.drop(columns=['event']).rename(columns=rename1)
ready2=struct201.drop(columns=['event']).rename(columns=rename2)

In [ ]:
#part1=mapfile.loc[mapfile['Short Name'].str.contains('ssaga_part101')]
intlist1=part1b.loc[part1b['Data Type']=='Integer']
#part2=mapfile.loc[mapfile['Short Name'].str.contains('ssaga_part201')]
intlist2=part2b.loc[part2b['Data Type']=='Integer']


In [ ]:
for i in intlist1['NDA Element Name']:
    try:
        ready1[i]=pd.to_numeric(ready1[i],errors='coerce').fillna(-9999).astype(int).astype(str).str.replace('-9999','')
    except:
        print('could not convert to int',i)

In [ ]:
for i in intlist2['NDA Element Name']:
    try:
        ready2[i]=pd.to_numeric(ready2[i],errors='coerce').fillna(-9999).astype(int).astype(str).str.replace('-9999','')
    except:
        print('could not convert to int',i)

In [ ]:
#ready1.ssaga_mj1a1
ssagas.mj1a1.value_counts()

In [ ]:
#Write out structure for validation
filePath=pathout+'/ssaga_part101.csv'
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("ssaga_part1,1\n")
    ready1.to_csv(f,index=False)

    
#Write out structure for validation
filePath=pathout+'/ssaga_part201.csv'
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("ssaga_part2,1\n")
    ready2.to_csv(f,index=False)

In [ ]:
omitthem=['ssaga_dp27_count']#,'ssaga_pt1_b1age','ssaga_pt16_check','ssaga_pt22_check','ssaga_tb4b_tally','ssaga_tb10c_days','ssaga_tb10c_tally','ssaga_tb16_tally','ssaga_al_total1','ssaga_al37_total','ssaga_al37d_ii','ssaga_al37d_iii','ssaga_al37d_iv','ssaga_al37d','ssaga_al_total2','ssaga_al_total3']
cleanupfuncs.redcleanup(structure="ssaga_part101", filePath="./prepped/hca/", extraomitlist=omitthem)

In [ ]:
#don't want to omit based on missing ness for part 2.  Want to omit based on missingness for part 1 after checking that there isn't any other data
one=pd.read_csv("./prepped/hca/ssaga_part101.csv",header=1,low_memory=False)[['src_subject_id', 'interview_date']]
two=pd.read_csv("./prepped/hca/ssaga_part201.csv",header=1,low_memory=False)[['src_subject_id', 'interview_date']]

In [ ]:
three=pd.merge(one[['src_subject_id','interview_date']],two[['src_subject_id','interview_date']],on=['src_subject_id','interview_date'],how='outer',indicator=True)
three.loc[~(three._merge=='both')]
three.columns
check=pd.merge(ready2,three.loc[~(three._merge=='both')],on=['src_subject_id','interview_date'])#,how='both')#
check.to_csv('whatsupwithssaga.csv',index=False)
#check.loc[check._merge=='left_only'].to_csv('whatsupwithssaga.csv',index=False)
check.columns

In [ ]:
#okay all good. 
#merge two with check to omit any in check
readyv2=pd.merge(ready2,check[['src_subject_id','interview_date']],on=['src_subject_id','interview_date'],how='outer',indicator=True)#
readyv2=readyv2.loc[readyv2._merge=='left_only'].drop(columns='_merge')#._merge.value_counts()
readyv2.shape
filePath=pathout+'/ssaga_part201.csv'
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("ssaga_part2,1\n")
    readyv2.to_csv(f,index=False)